In [ ]:
import re
import os
import cv2
import PIL
import math
import json
import base64
import numpy as np
from PIL import Image
from io import BytesIO
from llama_cpp import Llama
from rapidfuzz import fuzz , process
from typing import Any , List , Dict
from llama_cpp.llama_chat_format import Llava15ChatHandler

In [ ]:
def imagePath_to_base64(image_path):
    try:
        with Image.open(image_path) as img:
            if img.mode != "RGB":
                img = img.convert("RGB")
            buffer = BytesIO()
            img.save(buffer , format = "PNG")
            encode_string = base64.b64encode(buffer.getvalue()).decode("utf-8")
            return f"data:image/png;base64,{encode_string}"
    except FileExistsError:
        print("檔案不存在")
        return None
    except Exception:
        print("圖片處理錯誤")
        return None


In [ ]:
cell_box = "cell_box.json"
with open(cell_box, "r", encoding="utf-8") as f:
    cell_box = json.load(f)

html = "html.json"
with open(html, "r", encoding="utf-8") as f:
    html = json.load(f)


In [ ]:
html

In [ ]:

data = "test.json"
with open(data, "r", encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
model_path = "gemma-3-12b-it-q4_0.gguf"
mmproj_path = "mmproj-model-f16-12B.gguf"
chat_handler = Llava15ChatHandler(clip_model_path = mmproj_path)

llm = Llama(
    model_path = model_path,
    chat_handler = chat_handler,
    n_ctx = 2408 ,
    n_batch = 256,
    n_gpu_layers = -1,
    verbose = True,
    flash_attn = True
)

In [ ]:
image_path = "IMG.jpg"
image_uri= imagePath_to_base64(image_path)
if image_uri:
     prompt = f"""要求輸出（JSON-only）：
      你是點貨專家，OCR輸出位置訊息{cell_box}與文字內容{html}，請整理版面，輸出格式如下
      {{
    "PO單號": string|null,
    "品名與規格": [{{"品名": string|null ,"規格": string|null ,"數量" :string|null , "價錢":string|null}} ]
      }}
      僅輸出上述 JSON；禁止多餘文字、註解、程式碼區塊。"""



#      prompt = f"""要求輸出（JSON-only）：\n
#      你是點貨專家，OCR輸出{data}固定為'品名'+'規格'，請依照圖片格式依序將OCR的輸出結果填入，輸出格式如下\n
#      {{
#    "PO單號": string|null,
#    "品名與規格": [
#      {{"品名": string|null ,"規格": string|null ,"數量" :string|null , "價錢":string|null}}
#                  ]
#      }}
#      僅輸出上述 JSON；禁止多餘文字、註解、程式碼區塊。"""
    # 5. 建立對話訊息
    # 格式必須是 content 包含一個 text 和一個 image_url 的列表
messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {"type": "image_url", "image_url": {"url": image_uri}}
            ]
        }
    ]

    # 6. 進行推理
print("模型正在生成回應...")
response = llm.create_chat_completion(
    messages = messages,
    max_tokens = 2408,
    temperature = 0,
    )

    # 7. 顯示結果
answer = response['choices'][0]['message']['content']
print("="*30)
print(f"模型的回應：\n{answer}")
print("="*30)


In [ ]:
clean_answer = re.sub(r'```[a-zA-Z]*\n?', '', answer).strip()

In [ ]:
pred = json.loads(clean_answer)

In [ ]:
pred_list = pred["品名與規格"]

In [ ]:
pred_list

In [ ]:
db_table = list( [('調整螺絲組', 'ANBNH10-50'),
 ('調整螺絲', 'ANBNH8-45'),
 ('定位銷附螺牙', 'MSTG6-15'),
 ('調整螺絲組', 'ANBNM6-40'),
 ('六角支柱', 'C-PLSBRK14-100'),
 ('定位銷附螺牙', 'MSTG6-15'),
 ('定位銷附螺牙', 'MSTG6-20'),
 ('感應器用軌道', 'SENC32H-200-V15-W185'),
 ('調整螺絲組', 'ANBH6-40'),
 ('尼龍螺帽', 'HNTL1N-SUS-M10'),
 ('定位銷附螺牙', 'MSTG6-10'),
 ('定位銷附螺牙', 'MSTG6-15'),
 ('調整螺絲組', 'ANBH6-40'),
 ('六角支柱', 'C-PLSBRK8-30'),
 ('定位銷(g6)', 'MSTG5-10'),
 ('定位銷(g6)', 'MSTG5-15'),
 ('定位銷附螺牙', 'MSTG6-10')])

In [ ]:
db_table

In [ ]:
for item in pred["品名與規格"]:
    best_match = None
    best_score = -1

    for db_name, db_spec in db_table:
        # 直接計算品名與規格的 partial_ratio 分數並相加
        score = (fuzz.ratio(item["品名"], db_name) + fuzz.ratio(item.get("規格") or item.get("规格"), db_spec)) /2

        # 紀錄最高分
        if score > best_score:
            best_score = score
            best_match = (db_name, db_spec)

    print(f"查詢品項：{item}")
    print(f"最佳匹配：{best_match}，分數={best_score}")
    print("-" * 60)


查詢品項：{'品名': 'LS600-6機器人', '規格': '6KG 600MM IP54 220V'}
最佳匹配：('LS600-6 機器人', '6KG_600MM_IP54_220V')，分數=89.72431077694235
------------------------------------------------------------
查詢品項：{'品名': '新代-Battery_臺灣永士達', '規格': '3.6V'}
最佳匹配：('新代-Battery_台灣永士達', '3.6V')，分數=96.875
------------------------------------------------------------
查詢品項：{'品名': '軟體選配_增加1軸_自動化', '規格': '自動化擴充軸軟體選配'}
最佳匹配：('軟體選配_增加1軸_自動化', '自動化擴充軸軟體選配')，分數=100.0
------------------------------------------------------------
查詢品項：{'品名': 'SMD-30/30/30/30-XS四合一驅動器', '規格': '220V BAT'}
最佳匹配：('SMD-30/30/30/30-XS四合一驅動器', '220V/BAT')，分數=93.75
------------------------------------------------------------
查詢品項：{'品名': 'AMI電機', '規格': '額定1.27NM/最高5000RPM/帶鍵 槽/帶剎車'}
最佳匹配：('AM1電機', '额定1.27NM/最高5000RPM/带键槽')，分數=78.77551020408163
------------------------------------------------------------
查詢品項：{'品名': 'AMI電機', '規格': '額定1.27NM/最高5000RPM/帶鍵 槽'}
最佳匹配：('AM1電機', '额定1.27NM/最高5000RPM/带键槽')，分數=82.22222222222223
-----------------------------------------